<a href="https://colab.research.google.com/github/rk-ka/GDL_code_kuboko/blob/tensorflow_2/colab_03_03_vae_digits_train_OK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VAE Training

In [1]:
#!git clone -b tensorflow_2 --depth 1 https://github.com/davidADSP/GDL_code/
#%cd GDL_code

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

!ls /content/gdrive/MyDrive/GDL_code_my/

Mounted at /content/gdrive
callbacks_a.py	callbacks.py  loaders_a.py  loaders.py	__pycache__  test.py


In [4]:
#add project filepath to system path
import sys
sys.path.append('/content/gdrive/MyDrive/GDL_code_my')



In [5]:
!mkdir -p run/vae/
!ls run/vae/

In [6]:
%load_ext autoreload
%autoreload 2

## imports

In [7]:
import os
import time
import datetime

import tensorflow as tf
from tensorflow.keras import backend as K

from models.VAE_col import VariationalAutoencoder
from utils.loaders import load_mnist

In [8]:
# run params
SECTION = 'vae'
RUN_ID = '0002'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #
RUN_FOLDER

'run/vae/0002_digits'

## data

In [9]:
(x_train, y_train), (x_test, y_test) = load_mnist()

11493376/11490434 [==============================] - 1s 0us/step


## architecture

In [10]:
vae = VariationalAutoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
    , r_loss_factor = 1000
)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights_ch'))

In [11]:
vae.encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 28, 28, 32)   0           encoder_conv_0[0][0]             
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 14, 14, 64)   18496       leaky_re_lu[0][0]                
____________________________________________________________________________________________

In [12]:
vae.decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 3136)              9408      
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_1 (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 64)        0   

## training

In [13]:
LEARNING_RATE = 0.0005

In [14]:
vae.compile(LEARNING_RATE)

In [15]:
BATCH_SIZE = 32
EPOCHS = 100
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [16]:
start_time = datetime.datetime.now() #TIMER ON

vae.train(     
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

time_el =  datetime.datetime.now() - start_time #TIMER OFF
print ('Time elapsed: min {:.0f}, sec {:.0f}'.format(time_el.total_seconds() // 60, time_el.total_seconds() % 60 ))


Epoch 1/100
1871/1875 [============================>.] - ETA: 0s - loss: 58.7310 - reconstruction_loss: 55.5285 - kl_loss: 3.2025
Epoch 00001: saving model to run/vae/0002_digits/weights/weights.ckpt

Epoch 00001: saving model to run/vae/0002_digits/weights/weights_ch
1875/1875 [==============================] - 22s 12ms/step - loss: 58.7128 - reconstruction_loss: 55.5087 - kl_loss: 3.2040
Epoch 2/100
1871/1875 [============================>.] - ETA: 0s - loss: 51.7910 - reconstruction_loss: 47.8756 - kl_loss: 3.9154
Epoch 00002: saving model to run/vae/0002_digits/weights/weights.ckpt

Epoch 00002: saving model to run/vae/0002_digits/weights/weights_ch
1875/1875 [==============================] - 22s 12ms/step - loss: 51.7919 - reconstruction_loss: 47.8751 - kl_loss: 3.9168
Epoch 3/100
1871/1875 [============================>.] - ETA: 0s - loss: 50.3392 - reconstruction_loss: 46.1224 - kl_loss: 4.2168
Epoch 00003: saving model to run/vae/0002_digits/weights/weights.ckpt

Epoch 00003: 

Epoch 1/100
1871/1875 [============================>.] - ETA: 0s - loss: 58.7310 - reconstruction_loss: 55.5285 - kl_loss: 3.2025
Epoch 00001: saving model to run/vae/0002_digits/weights/weights.ckpt

Epoch 00001: saving model to run/vae/0002_digits/weights/weights_ch
1875/1875 [==============================] - 22s 12ms/step - loss: 58.7128 - reconstruction_loss: 55.5087 - kl_loss: 3.2040
Epoch 2/100
1871/1875 [============================>.] - ETA: 0s - loss: 51.7910 - reconstruction_loss: 47.8756 - kl_loss: 3.9154
Epoch 00002: saving model to run/vae/0002_digits/weights/weights.ckpt

Epoch 00002: saving model to run/vae/0002_digits/weights/weights_ch
1875/1875 [==============================] - 22s 12ms/step - loss: 51.7919 - reconstruction_loss: 47.8751 - kl_loss: 3.9168
Epoch 3/100
1871/1875 [============================>.] - ETA: 0s - loss: 50.3392 - reconstruction_loss: 46.1224 - kl_loss: 4.2168
Epoch 00003: saving model to run/vae/0002_digits/weights/weights.ckpt

Epoch 00003: saving model to run/vae/0002_digits/weights/weights_ch
1875/1875 [==============================] - 22s 12ms/step - loss: 50.3366 - reconstruction_loss: 46.1197 - kl_loss: 4.2169
Epoch 4/100
1875/1875 [==============================] - ETA: 0s - loss: 49.3391 - reconstruction_loss: 44.9397 - kl_loss: 4.3993

- - -

Epoch 00097: saving model to run/vae/0002_digits/weights/weights_ch
1875/1875 [==============================] - 22s 12ms/step - loss: 43.5247 - reconstruction_loss: 38.0446 - kl_loss: 5.4801
Epoch 98/100
1875/1875 [==============================] - ETA: 0s - loss: 43.5115 - reconstruction_loss: 38.0346 - kl_loss: 5.4769
Epoch 00098: saving model to run/vae/0002_digits/weights/weights.ckpt

Epoch 00098: saving model to run/vae/0002_digits/weights/weights_ch
1875/1875 [==============================] - 22s 12ms/step - loss: 43.5101 - reconstruction_loss: 38.0332 - kl_loss: 5.4769
Epoch 99/100
1875/1875 [==============================] - ETA: 0s - loss: 43.5124 - reconstruction_loss: 38.0408 - kl_loss: 5.4716
Epoch 00099: saving model to run/vae/0002_digits/weights/weights.ckpt

Epoch 00099: saving model to run/vae/0002_digits/weights/weights_ch
1875/1875 [==============================] - 22s 12ms/step - loss: 43.5122 - reconstruction_loss: 38.0404 - kl_loss: 5.4718
Epoch 100/100
1873/1875 [============================>.] - ETA: 0s - loss: 43.4941 - reconstruction_loss: 38.0205 - kl_loss: 5.4736
Epoch 00100: saving model to run/vae/0002_digits/weights/weights.ckpt

Epoch 00100: saving model to run/vae/0002_digits/weights/weights_ch
1875/1875 [==============================] - 22s 12ms/step - loss: 43.4967 - reconstruction_loss: 38.0232 - kl_loss: 5.4735
Time elapsed: min 37, sec 13

In [17]:
#copy to my project dir
#cp -R <source_folder> <destination_folder>
!cp -R run/vae/ /content/gdrive/MyDrive/GDL_code_my